In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')

games_df = pd.read_csv("games_dataset.csv")
# games_json = pd.read_json("games_dataset.json")
# 1. Drop irrelevant columns 
# 2. Trim rows with Recommendations = 0
# 3. Columns to keep:
# | name | 

In [9]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# games_df
# games_json
# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_rows')

In [73]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# row = games_df.loc[games_df["Name"] == "ELDEN RING"]
# row

In [2]:
print(games_df.info())
# print("\n\n\n")
print(games_df.describe())

# print(movies.describe())
# print(movies.info())
# print(movies.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85103 entries, 0 to 85102
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       85103 non-null  int64  
 1   Name                        85097 non-null  object 
 2   Release date                85103 non-null  object 
 3   Estimated owners            85103 non-null  object 
 4   Peak CCU                    85103 non-null  int64  
 5   Required age                85103 non-null  int64  
 6   Price                       85103 non-null  float64
 7   DLC count                   85103 non-null  int64  
 8   About the game              81536 non-null  object 
 9   Supported languages         85103 non-null  object 
 10  Full audio languages        85103 non-null  object 
 11  Reviews                     9743 non-null   object 
 12  Header image                85103 non-null  object 
 13  Website                     394

In [75]:
# i0 = 0
# i1 = 0
# i5 = 0
# i10 = 0
# i100 = 0
# for x in games_df["Recommendations"]:
#     print(x)
#     if x == 0:
#         i0 += 1
#     if x > 1:
#         i1 += 1
#     if x > 5:
#         i5 += 1
#     if x > 10:
#         i10 += 1
#     if x > 100:
#         i100 += 1
    
# print(f"i0: {i0}\ni1: {i1}\ni5: {i5}\ni10: {i10}\ni100: {i100}")

In [76]:
# Displays non-null values, used to trim/reduce data
# games_df.info()

# Lowercase everything
games_df.columns = games_df.columns.str.lower()
games_df = games_df.map(lambda s: s.lower() if type(s) == str else s)

In [77]:
games_df
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

# print(games_df)

,appid,name,release date,estimated owners,peak ccu,required age,price,dlc count,about the game,supported languages,...,average playtime two weeks,median playtime forever,median playtime two weeks,developers,publishers,categories,genres,tags,screenshots,movies
0,20200,galactic bowling,"oct 21, 2008",0 - 20000,0,0,19.99,0,galactic bowling is an exaggerated and stylize...,['english'],...,0,0,0,perpetual fx creative,perpetual fx creative,"single-player,multi-player,steam achievements,...","casual,indie,sports","indie,casual,sports,bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,train bandit,"oct 12, 2017",0 - 20000,0,0,0.99,0,the law!! looks to be a showdown atop a train....,"['english', 'french', 'italian', 'german', 'sp...",...,0,0,0,rusty moyher,wild rooster,"single-player,steam achievements,full controll...","action,indie","indie,action,pixel graphics,2d,retro,arcade,sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,jolt project,"nov 17, 2021",0 - 20000,0,0,4.99,0,jolt project: the army now has a new robotics ...,"['english', 'portuguese - brazil']",...,0,0,0,campião games,campião games,single-player,"action,adventure,indie,strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,henosis™,"jul 23, 2020",0 - 20000,0,0,5.99,0,henosis™ is a mysterious 2d platform puzzler w...,"['english', 'french', 'italian', 'german', 'sp...",...,0,0,0,odd critter games,odd critter games,"single-player,full controller support","adventure,casual,indie","2d platformer,atmospheric,surreal,mystery,puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,two weeks in painland,"feb 3, 2020",0 - 20000,0,0,0.00,0,about the game play as a hacker who has arrang...,"['english', 'spanish - spain']",...,0,0,0,unusual games,unusual games,"single-player,steam achievements","adventure,indie","indie,adventure,nudity,violent,sexual content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85098,2669080,mannerheim's saloon car,"jan 2, 2024",0 - 0,0,0,0.00,0,marshal mannerheim’s saloon car is the train c...,"['english', 'finnish']",...,0,0,0,xamk game studios,"sodan ja rauhan keskus muisti, päämajamuseo","single-player,tracked controller support,vr only","adventure,simulation",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
85099,2736910,beer run,"jan 3, 2024",0 - 0,0,0,0.00,0,beer run is an indie game created to steal bee...,['english'],...,0,0,0,955 games,955 games,single-player,"casual,indie",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
85100,2743220,my friend the spider,"jan 4, 2024",0 - 0,0,0,0.00,0,a small 'horror' narrative game about isolatio...,['english'],...,0,0,0,mca,mca,single-player,"adventure,simulation",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
85101,2293130,path of survivors,"jan 8, 2024",0 - 0,0,0,3.99,0,path of survivors is a multi-class auto-battle...,['english'],...,0,0,0,limited input,limited input,"single-player,steam achievements,partial contr...","action,casual,indie,rpg,simulation",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [78]:
# FILTER OUT 0 RECOMMENDATIONS
games_df = games_df[games_df["recommendations"] > 0]
# games_df.reset_index(drop=True, inplace=True)

games_df

,appid,name,release date,estimated owners,peak ccu,required age,price,dlc count,about the game,supported languages,...,average playtime two weeks,median playtime forever,median playtime two weeks,developers,publishers,categories,genres,tags,screenshots,movies
10,1026420,warsaw,"oct 2, 2019",20000 - 50000,5,0,23.99,0,use everything at your disposal to help a team...,"['english', 'french', 'german', 'polish', 'rus...",...,0,93,0,pixelated milk,"pixelated milk,gaming company","single-player,steam achievements,steam trading...","indie,rpg","tactical rpg,turn-based strategy,wargame,histo...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
15,22670,alien breed 3: descent,"nov 17, 2010",200000 - 500000,3,0,9.99,0,alien breed™ 3: descent is the final explosive...,"['english', 'french', 'german', 'italian', 'ja...",...,0,24,0,team17 digital ltd,team17 digital ltd,"single-player,multi-player,co-op,steam achieve...",action,"action,shooter,sci-fi,aliens,third person,isom...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN
17,346560,hero of the kingdom ii,"feb 20, 2015",100000 - 200000,5,0,7.99,0,sail to the farthest islands to save your sist...,"['english', 'french', 'german', 'spanish - spa...",...,0,303,0,lonely troops,lonely troops,"single-player,steam achievements,steam trading...","adventure,casual,indie,rpg","adventure,casual,point & click,rpg,indie,isome...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
18,1097880,super naughty maid 2,"jun 20, 2019",0 - 20000,4,0,9.59,2,"story leona, your cute and lewd maid is back f...","['english', 'traditional chinese']",...,0,30,0,kenzsoft,denpasoft,"single-player,steam trading cards","casual,indie",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
22,434030,aerofly fs 2 flight simulator,"nov 20, 2017",100000 - 200000,19,0,37.49,23,aerofly fs 2 lets you explore the world of fly...,"['english', 'german', 'french']",...,0,127,0,ipacs,ipacs,"single-player,partial controller support","action,indie,racing,simulation","flight,simulation,vr,racing,physics,open world...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84532,2458490,fallen shinobi,"dec 18, 2023",0 - 20000,492,0,13.49,0,it is an era of warring states. by some good f...,"['japanese', 'english', 'simplified chinese', ...",...,0,0,0,marron☆marron,mango party,single-player,simulation,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
84709,2712540,完蛋！我也能追到美女了！,"dec 28, 2023",0 - 0,0,0,0.89,0,你曾经梦想过在电影或电视剧中体验恋爱生活吗？ 想感受主角的喜怒哀乐，参与改变剧情走向吗？ 从...,['simplified chinese'],...,0,0,0,火花工作室,"火花工作室,钢爪工作室","single-player,steam achievements,steam cloud","casual,rpg,simulation",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
84869,2495450,winter memories,"jan 5, 2024",0 - 0,0,0,15.99,0,after the unforgettable summer memories you’ve...,"['english', 'japanese', 'simplified chinese']",...,0,0,0,dojin otome,kagura games,"single-player,steam cloud","adventure,casual,rpg,simulation",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
84963,2487350,alex jones: nwo wars,"jan 3, 2024",0 - 20000,63,0,17.76,0,play the official alex jones video game! save ...,['english'],...,0,0,0,nwo wars team,nwo wars team,"single-player,full controller support,steam cloud","action,indie","action,arcade,shooter,2d platformer,shoot 'em ...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [79]:
# # ======================================================
# # ======== FORMAT RELEASE DATE "MMM DD, YYYY" ==========
# # ======================================================
# len_11 = 0
# len_12 = 0
# len_unknown = 0

# for i, date in enumerate(games_df["release date"]):
#     if len(date) in [11]: 
#         games_df.loc[i, "release date"] = games_df.loc[i, "release date"][0:3] + " 0" + games_df.loc[i, "release date"][4:11]
#         len_11 += 1
        
#     elif len(date) in [12]: 
#         len_12 += 1
#     else:
#         games_df.loc[i, "release date"] = games_df.loc[i, "release date"][0:3] + " 01, " + games_df.loc[i, "release date"][4:8]
#         # print(date) 
#         len_unknown += 1

#     # if games_df["Supported languages"]


# print(len_11, len_12, len_unknown)

# # ===========================================================
# # ======== SORT BY RELEASE DATE YYYY-MM-DD==========
# # ===========================================================

# # Convert to class datetime.date > YYYY-MM-DD HH:MM:SS
# # games_df["release date"] = pd.to_datetime(games_df["release date"])
# games_df = games_df.copy()
# games_df["release date"] = pd.to_datetime(games_df["release date"], format="%b %d, %Y")

# # Sort data by descending Release date
# games_sorted = games_df.sort_values(by=["release date"], ascending=False)

# # Remove irrelevant timestmap
# # games_sorted["Release date"] = games_sorted["Release date"].dt.date

# games_sorted

In [80]:
# DROPPING ALL COLUMNS EXCEPT FOR:

keep_col_1 = ["name", "release date", "about the game", "supported languages", "windows",
             "categories", "genres", "tags"]

keep_col_2 = keep_col_1 + ["reviews", "metacritic score", "positive", "negative", "recommendations", "developers", "publishers"]


for col in games_df.columns:
    if col not in keep_col_1:
        games_df.drop(columns=[col], inplace=True)

games_df

C:\Users\shenje1\AppData\Local\Temp\ipykernel_30944\1731963190.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_df.drop(columns=[col], inplace=True)
C:\Users\shenje1\AppData\Local\Temp\ipykernel_30944\1731963190.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_df.drop(columns=[col], inplace=True)
C:\Users\shenje1\AppData\Local\Temp\ipykernel_30944\1731963190.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games

,name,release date,about the game,supported languages,windows,categories,genres,tags
10,warsaw,"oct 2, 2019",use everything at your disposal to help a team...,"['english', 'french', 'german', 'polish', 'rus...",True,"single-player,steam achievements,steam trading...","indie,rpg","tactical rpg,turn-based strategy,wargame,histo..."
15,alien breed 3: descent,"nov 17, 2010",alien breed™ 3: descent is the final explosive...,"['english', 'french', 'german', 'italian', 'ja...",True,"single-player,multi-player,co-op,steam achieve...",action,"action,shooter,sci-fi,aliens,third person,isom..."
17,hero of the kingdom ii,"feb 20, 2015",sail to the farthest islands to save your sist...,"['english', 'french', 'german', 'spanish - spa...",True,"single-player,steam achievements,steam trading...","adventure,casual,indie,rpg","adventure,casual,point & click,rpg,indie,isome..."
18,super naughty maid 2,"jun 20, 2019","story leona, your cute and lewd maid is back f...","['english', 'traditional chinese']",True,"single-player,steam trading cards","casual,indie",NaN
22,aerofly fs 2 flight simulator,"nov 20, 2017",aerofly fs 2 lets you explore the world of fly...,"['english', 'german', 'french']",True,"single-player,partial controller support","action,indie,racing,simulation","flight,simulation,vr,racing,physics,open world..."
...,...,...,...,...,...,...,...,...
84532,fallen shinobi,"dec 18, 2023",it is an era of warring states. by some good f...,"['japanese', 'english', 'simplified chinese', ...",True,single-player,simulation,NaN
84709,完蛋！我也能追到美女了！,"dec 28, 2023",你曾经梦想过在电影或电视剧中体验恋爱生活吗？ 想感受主角的喜怒哀乐，参与改变剧情走向吗？ 从...,['simplified chinese'],True,"single-player,steam achievements,steam cloud","casual,rpg,simulation",NaN
84869,winter memories,"jan 5, 2024",after the unforgettable summer memories you’ve...,"['english', 'japanese', 'simplified chinese']",True,"single-player,steam cloud","adventure,casual,rpg,simulation",NaN
84963,alex jones: nwo wars,"jan 3, 2024",play the official alex jones video game! save ...,['english'],True,"single-player,full controller support,steam cloud","action,indie","action,arcade,shooter,2d platformer,shoot 'em ..."


In [81]:
games_df_backup = games_df.copy()

In [82]:
games_df = games_df_backup

In [83]:
# # Create rating column and drop positive and negative

# games_sorted["rating"] = np.where(
#     (games_sorted["positive"] == 0) & (games_sorted["negative"] == 0), 
#     0.5, 
#     games_sorted["positive"] / (games_sorted["positive"] + games_sorted["negative"])
# )
# games_sorted = games_sorted.drop(columns=["positive", "negative"])

# games_backup = games_sorted

In [84]:
# games_reduced = games_sorted[games_sorted["release date"] >= pd.to_datetime("2023-01-01")]
games_df = games_df[games_df["windows"] == True]
games_df = games_df[games_df["supported languages"].apply(lambda x: "english" in x)]

games_df["name"] = games_df["name"].str.replace("™", "", regex=False)
games_df["name"] = games_df["name"].str.replace(r"[^\w\s']+", '', regex=True)


In [85]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13356 entries, 10 to 85086
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 13356 non-null  object
 1   release date         13356 non-null  object
 2   about the game       13331 non-null  object
 3   supported languages  13356 non-null  object
 4   windows              13356 non-null  bool  
 5   categories           13283 non-null  object
 6   genres               13338 non-null  object
 7   tags                 12870 non-null  object
dtypes: bool(1), object(7)
memory usage: 847.8+ KB


In [86]:


# find_null_col = ["about the game", "reviews", "tags", "genres"]
# for col in find_null_col:
#     games_reduced = games_reduced.dropna(subset=[col]) 
    
games_df = games_df.dropna(subset=["about the game"])           
# games_df = games_df.dropna(subset=["genres"])           
# games_df = games_df.dropna(subset=["categories"])           
# games_df = games_df.dropna(subset=["tags"])           
# games_reduced = games_reduced.dropna(subset=["reviews"])           
# games_reduced = games_reduced.dropna(subset=["about the game", "reviews", "tags", "genres"])           
# games_reduced = games_reduced.dropna(subset=["about the game", "reviews", "tags", "genres"])           

# len(games_reduced)
# games_reduced.info()

# DEFINITE FEATURES: 
# name, release date, about the game

# games_reduced = games_reduced[["name", "about the game", "genres"]]
games_df["feature"] = games_df["about the game"] + games_df["genres"] + games_df["categories"] + games_df["tags"]
games_df = games_df.drop(columns=["about the game", "genres", "categories", "tags", "windows", "supported languages"])

games_df.reset_index(drop=True, inplace=True)

games_df


,name,release date,feature
0,warsaw,"oct 2, 2019",use everything at your disposal to help a team...
1,alien breed 3 descent,"nov 17, 2010",alien breed™ 3: descent is the final explosive...
2,hero of the kingdom ii,"feb 20, 2015",sail to the farthest islands to save your sist...
3,super naughty maid 2,"jun 20, 2019",NaN
4,aerofly fs 2 flight simulator,"nov 20, 2017",aerofly fs 2 lets you explore the world of fly...
...,...,...,...
13326,world warfare economics,"dec 20, 2023",NaN
13327,fallen shinobi,"dec 18, 2023",NaN
13328,winter memories,"jan 5, 2024",NaN
13329,alex jones nwo wars,"jan 3, 2024",play the official alex jones video game! save ...


In [87]:
# cv = CountVectorizer(max_features=100000, stop_words="english")
cv = CountVectorizer()

In [88]:
vector = cv.fit_transform(games_df["feature"].values.astype("U")).toarray()

In [89]:
# vector.shape # (row, col) => (13383, 10000)
cv.get_feature_names_out()

array(['00', '000', '0000', ..., '𝗗𝗲𝘃𝗲𝗹𝗼𝗽𝗺𝗲𝗻𝘁', '𝗦𝗾𝘂𝗶𝗿𝗿𝗲𝗹', '𝗧𝗲𝗮𝗺'],
      dtype=object)

In [95]:
def compute_cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

In [104]:
vec1 = vector[0]
vec2 = vector[1]
similarity = compute_cosine_similarity(vec1, vec2)
np.set_printoptions(threshold=np.inf)
print(len(vector[10]))
np.set_printoptions(threshold=1000)

68017


In [90]:
similarity = cosine_similarity(vector)

In [91]:
def recommend_cs(game):
    index = games_df[games_df["name"] == game].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
    cnt = 1
    for i in distance[0:11]:
        if i[0] == index:
            print(f"Top 10 games similar to: {games_df.iloc[i[0]]["name"]}")
            continue
        print(f"[{cnt}]:\t{games_df.iloc[i[0]]["name"]}")
        cnt += 1
        

In [92]:
distance = euclidean_distances(vector)


In [93]:

def recommend_ed(game):
    index = games_df[games_df["name"] == game].index[0]
    distances = list(enumerate(distance[index]))
    sorted_distances = sorted(distances, key=lambda x: x[1])
    cnt = 1
    for i in sorted_distances[0:11]:  
        if i[0] == index:
            print(f"Top 10 games similar to: {games_df.iloc[i[0]]["name"]}")   
            continue     
        print(f"[{cnt}]:\t{games_df.iloc[i[0]]["name"]}")
        # print(games_df.iloc[i[0]]["name"])
        cnt += 1

In [94]:
# for x in games_reduced["name"]:
#     print(x)

recommend_cs("elden ring")
print("@@@@@@@@@@@@@@@@")
recommend_ed("elden ring")


Top 10 games similar to: elden ring
[1]:	mortal online 2
[2]:	lords of the fallen
[3]:	ace of seafood
[4]:	conan exiles
[5]:	lost ark
[6]:	ghostlore
[7]:	brink
[8]:	brink
[9]:	brink
[10]:	bless unleashed
@@@@@@@@@@@@@@@@
Top 10 games similar to: elden ring
[1]:	sword art online lost song
[2]:	grounded
[3]:	the elder scrolls online
[4]:	bound by flame
[5]:	warhammer vermintide 2
[6]:	rio  raised in oblivion
[7]:	terraria
[8]:	warhammer 40000 eternal crusade
[9]:	dissidia final fantasy nt free edition
[10]:	assassin's creed unity


In [105]:
# create exampele 3D plots of cosine similarity and euclidean distance